# Worksheet 05

Name:  Shengduo Li
UID: U64101332

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

[0, .5] and [1.5, 2, 6, 6.5, 7]     mean1=.25 mean2=4.6 

[0, .5, 1.5, 2] and [6, 6.5, 7]     mean1'=1 mean2'=6.5

[0, .5, 1.5, 2] and  [6, 6.5, 7]     centroids does not change


b) Describe in plain english what the cost function for k means is.

 The cost function is a measurement for the spread of the clusters. We want it to be minimized when cluster the dataset.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Since each time we start randomly to pick k centers, so the k centers might be different each time.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, it always converge. Since the dataset is finite and the number of partition is also finite, so there is only finite number of cost function values.

e) Follow along in class the implementation of Kmeans

In [2]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)),self.k,replace=False)]
    
    def distance(self,x,y):
        return np.linalg.norm(x - y)
    
    def is_diff_clusters(self,centers,new_centers):
        n = len(centers)
        for i in range(n):
            if centers[i][0]!=new_centers[i][0]:
                return True
        return False
    
    def assign(self,centers):
        for i in range(len(self.data)):
            #minimum=self.distance(centers[0],self.data[0])
            #self.assignment[i]=0
            delta=[float('inf'),0]
            for j in range(len(centers)):
                dist=self.distance(centers[j],self.data[i])
                if dist < delta[0]:
                    #minimum = dist
                    #self.assignment[i] = j
                    delta[0]=dist
                    delta[1]=j
            self.assignment[i]=delta[1]        
        return
    
    def get_centers(self):
        new_centers=[]
        for i in set(self.assignment):
            cluster=[]
            for j in range(len(self.data)):
                if self.assignment[j]==i:
                    cluster.append(self.data[j])
            x=0
            y=0
            for delta in range(len(cluster)):
                x+=cluster[delta][0]
                y+=cluster[delta][1]
            centers.append([x/len(cluster), y/len(cluster)])
            #new_centers.append(np.mean(cluster))
            #print(np.array(new_centers))
        return np.array(new_centers)

    def lloyds(self):
        centers=self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers=self.get_centers()
        while self.is_diff_clusters(centers,new_centers):
            #new_centers=self.get_centers()
            self.assign(new_centers)
            centers=new_centers
            self.snap(centers)
            new_centers=self.get_centers()

        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

ModuleNotFoundError: No module named 'PIL'